In [2]:
class Vehicle():
    def __init__(self, id):
        self.id = id
        self.history = []
        self.batch_num = 0
        self.last_avail_time = 0

    def bind_fn(self, fx): #Fx is expected to be of type f()(current_loc, current_time, start_loc, start_time, end_loc). Assumes the data set does not contain self impossible rides, ie, that if you start a ride on time you can finish it as well
        self.fn = fx

    def getEndTime(self, start_time, end_loc):
        return abs(end_loc[0] - start_loc[0]) + abs(end_loc[1] - start_loc[1]) + start_time - 1

    def bind(self, batch, ride): #Probably don't need all this info but it is nice to keep track of. End time here will be the ACTUAL finish time of the vehicle, as will start time, capped to a min of ride_start_time. Start time is the delta step when the vehicle is allowed to make its first move
        self.ride = ride[-1]
        self.history.append(ride_id)
        self.start_loc = (ride[0], ride[1])
        self.end_loc = (ride[2], ride[3])
        self.start_time = (ride[4] if self.last_avail_time < ride[1][4] else self.last_avail_time)
        self.end_time = self.getEndTime(self, self.start_time, self.end_loc)
        self.last_avail_time = self.end_time + 1
        self.batch = batch
        self.batch_num += 1 #Used to externally slice the data set to pass proper batch reference to bind()


    def score(self):
        try:
            tmp = [self.fn(self.end_loc, self.end_time, e[1], e[2], e[3]) for e in batch] #Batch is assumed to be a list of tuples of type (ride_id, start_loc, start_time, end_loc, etc...)
            idx = max(range(len(tmp)), key=tmp.__getitem__)
            ret = batch[idx]
            del(batch[idx])
            return ret
        except AttributeError:
            print("Missing a bind")
            return None
        except ValueError: #max(empty), batch is depleted. This can only happen in the very last batch when the total number of rides isn't evenly divded by the number of vehicles.
            return None




In [3]:
#Some example functions to calculate score
import math

def getDist(current_loc, start_loc):
    return abs(start_loc[0] - current_loc[0]) + abs(start_loc[1] - current_loc[1])

def fn_bonus_greed_only(current_loc, current_time, start_loc, start_time, end_loc):
    #Check to see if the ride is actually possible to start on time
    if getDist(current_loc, start_loc) + current_time  > start_time:
        return 0
    return getDist(start_loc, end_loc) #No need to add bonus to all values

def fn_ratio_greed(current_loc, current_time, start_loc, start_time, end_loc):
    return (getDist(start_loc, end_loc) + (2 if getDist(current_loc, start_loc) + current_time > start_time else 0 )/getDist(current_loc, start_loc)

def fn_log_greed(current_loc, current_time, start_loc, start_time, end_loc):
    return math.log(getDist(start_loc, end_loc) + (2 if getDist(current_loc, start_loc) + current_time > start_time else 0 ),getDist(current_loc, start_loc))

SyntaxError: invalid syntax (<ipython-input-3-4f420f87ae0d>, line 16)